<a href="https://colab.research.google.com/github/sirzzang/TIL_multicampus_lecturefile/blob/master/%5B20200117%5Dtensorflow_CNN_MNIST_ensemble_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **OOP로 ensemble 구현**
---
> 일단 kaggle 말고 tensorflow MNIST 데이터 사용
---
> *OOP 구현*에서 주의할 점
---
1. 구조를 먼저 생각하고, class 안에 변수와 함수 덩어리를 넣어 놓자.
    * class 안에 넣어 놓아야 하는지, class 밖에서 호출해야 하는지.
2. 각 구조별로 객체가 생성되는지 확인하는 과정을 거치자.
    * 예를 들어, `print("객체가 생성되었어요.")`.
    * console 통해 제대로 실행되는지 확인하며 단계별로 진행.
3. 객체 생성 시 코드 작성 순서도 매우 중요하다.
4. 딥러닝 코드로 구현하는 과정 자체는, 이전의 코드를 그대로 넣으면 된다.
    * `self` : class 안에서 계속 사용해야 하는 변수인 경우, self 붙임.
    * 외부에 노출할 필요가 없는 변수인 경우, `self` 붙이지 않음.

---
> *ensemble model 구현*에서 주의할 점
1. ensemble은 여러 개의 CNN 모델을 만들어서 이용한다.
    * list 이용 : 각 개체에 CNN 모델을 하나씩 저장.
2. ensemble이 아닐 경우, 정확도 구하는 함수, 예측하는 함수 모두 class 안에 정의해도 된다. 다만, 지금은 **ensemble이기 때문에** Hval 구하는 부분까지만 class 안에 정의한다.

In [0]:
# drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# module import

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd

In [0]:
# graph 초기화

tf.reset_default_graph()

### Step 1. Class 만들기

#### 1.1. 생성자
* 처음에 모델 실행하자마자 뭘 할지 고민.
* 변수 생성, 변수로 받아올 인자 정의해 두기.
* 실행하자 마자 어떤 함수 실행하는지 호출하기.

#### 1.2. 그래프 그리기
* 계속 사용하는 변수에 `self`.
    > * self 안 붙으면 scope. 변수 가동 범위가 해당 함수 내로 국한됨. 지역변수(local variable) 개념.
    > * 제대로 안 붙이면 오류(NameError: name 'drop_rate' is not defined)
* tensorflow node를 만드는 과정 자체가 그래프를 그리는 과정.
* node : cost, train까지 만들면 tensorflow의 전체 그래프가 완성된다.

#### 1.3. 학습시키기

#### 1.4. 실행값 구하기

#### 1.5. 기타: 만약 ensemble이 아니라면, 정확도 구하는 기능, 예측값 구하는 기능 모두 다 넣는다.


In [0]:
# 1. 모델 객체를 class로 정의

class CnnModel: 

    # 1.1. 생성자

    def __init__(self, session, data):
        print("객체 생성") # 실행되는지 확인하기 위함.
        self.sess = session # session 인자로 받아서 넘겨줄 변수.
        self.mnist = data # data로 사용할 변수.
        self.build_graph() # 객체 생성되면서 바로 그래프를 그린다.
 
    # 1.2. 그래프 그리는 기능 : 객체 생성되자마자 그래프 노트 필요하므로, 생성자에 포함.
    # self 붙는 변수에 주의.

    def build_graph(self):
        print("그래프 그려")

        # 1) placeholder
        self.X = tf.placeholder(shape=[None,784], dtype=tf.float32) # 외부 노출 = 계속해서 사용.
        self.Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
        self.drop_rate = tf.placeholder(dtype=tf.float32)
        
        # 2) convolution layer
        x_img = tf.reshape(self.X, [-1,28,28,1])
        
        L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu)
        L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2], padding="SAME", strides=2)
        L1 = tf.layers.dropout(inputs=L1, rate=self.drop_rate)
        
        L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu)
        L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2], padding="SAME", strides=2)
        L2 = tf.layers.dropout(inputs=L2, rate=self.drop_rate)
        
        L2 = tf.reshape(L2, [-1, 7*7*64])
        
        dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
        dense1 = tf.layers.dropout(inputs=dense1, rate=self.drop_rate)
        
        dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
        dense2 = tf.layers.dropout(inputs=dense2, rate=self.drop_rate)
        
        dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
        dense3 = tf.layers.dropout(inputs=dense3, rate=self.drop_rate)
 
        self.H = tf.layers.dense(inputs=dense3, units=10)

        # 3) cost     
        self.cost = tf.losses.softmax_cross_entropy(self.Y, self.H) 
        
        self.train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)
 
    # 1.3. 학습시키는 기능: 학습은 class 밖에서 진행하므로, 생성자 안에 포함하지 않음.
    def train_graph(self):
        self.sess.run(tf.global_variables_initializer())
        print("텐서플로우 그래프 학습")

        num_of_epoch = 3 # 외부에 노출할 필요 없는 변수.
        batch_size = 100
        
        for step in range(num_of_epoch):
            num_of_iter = int(self.mnist.train.num_examples / batch_size)
            cost_val = 0
            for i in range(num_of_iter):
                batch_x, batch_y = self.mnist.train.next_batch(batch_size)
                _, cost_val = self.sess.run([self.train, self.cost], feed_dict = {self.X :batch_x,
                                                                                  self.Y : batch_y,
                                                                                  self.drop_rate : 0.4})
            if step % 3 == 0:
                print("cost : {}".format(cost_val))
 
    # 1.4. H 실행시킨 값 구하는 기능
    def get_Hval(self):
        print("입력한 값에 대한 H를 리턴해요!")
        # 테스트 데이터에 대한 예측값(H)을 구해서 리턴해요!
        return self.sess.run(self.H, feed_dict={self.X : self.mnist.test.images,
                                                self.drop_rate : 0})

### Step 2. 모델 객체 만들기
* 여러 개의 모델을 그 자체로 객체로 잡아서 만든다. 즉, 모델의 틀을 만들어 놓는 과정이라고 생각하자.
* 모델 개수가 곧 객체의 개수라고 생각.
* list 사용: 아래와 같은 방식은 매우 노동집약적.
    ```
    model1 =CnnModel()
    model2 =CnnModel()
    ...
    ```
* 모델 들어 있는 list 실행 결과
```
[<__main__.CnnModel object at 0x7f858574c7f0>, <__main__.CnnModel object at 0x7f858574c860>, <__main__.CnnModel object at 0x7f858574c748>, <__main__.CnnModel object at 0x7f858574c828>, <__main__.CnnModel object at 0x7f858574c908>, <__main__.CnnModel object at 0x7f858574ca90>, <__main__.CnnModel object at 0x7f858574cb00>, <__main__.CnnModel object at 0x7f858574c0f0>, <__main__.CnnModel object at 0x7f858574cc88>, <__main__.CnnModel object at 0x7f858574cc50>]
```
* session 정의
    * 그래프 학습시 session이 있어야 학습할 수 있음.
    * session 변수 여러 개 있을 필요 없음(메모리 낭비) .
    * 모델 객체 만들 때 한 번 만들어서 실행 시 넘겨주도록 함.

* data 변수 정의

In [6]:
# 2. 모델 객체 만들기
# 모델 객체를 만들고, 그것을 list 안에 넣어 활용한다.
# 여러 번 정의할 필요 없는 data, session 변수 정의하고, 위의 셀에 넘겨준다.
mnist = input_data.read_data_sets("/content/drive/My Drive/data/mnist", one_hot = True)
sess = tf.Session()
num_of_model = 2 # 외부 노출 필요 없음. 연습용으로 2개의 모델 생성할 것.
models = [CnnModel(sess, mnist) for x in range(num_of_model)]

Extracting /content/drive/My Drive/data/mnist/train-images-idx3-ubyte.gz
Extracting /content/drive/My Drive/data/mnist/train-labels-idx1-ubyte.gz
Extracting /content/drive/My Drive/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /content/drive/My Drive/data/mnist/t10k-labels-idx1-ubyte.gz
객체 생성
그래프 그려
객체 생성
그래프 그려


### Step 3. 모델 학습시키기
* 모델 만들었으니까, 각 모델을 학습시켜야 한다.
* 학습시킬 때마다 각각 다른 모델이 완성되어야 한다.
* 모델에 대해 위의 class에서 정의한 train_graph 함수 호출한다.

In [8]:
# 3. 모델 학습시키기
# 모델을 학습해보아요!
for i in range(num_of_model):
    models[i].train_graph()

print(models)

텐서플로우 그래프 학습
cost : 0.03126775100827217
텐서플로우 그래프 학습
cost : 0.11475653946399689
[<__main__.CnnModel object at 0x7f5c2410f940>, <__main__.CnnModel object at 0x7f5c22e067b8>]


### Step 4. H값 구하기

* 모델 만들었으니까, 각 모델을 학습시켜야 한다.
* 학습시킬 때마다 각각 다른 모델이 완성되어야 한다.
* 모델에 대해 위의 class에서 정의한 train_graph 함수 호출한다.

In [9]:
# test data 넣은 것에 대해 H값을 구해보아요!
initial_predict = np.zeros([mnist.test.num_examples,10]) 

for i in range(num_of_model):
    initial_predict = initial_predict + models[i].get_Hval()
print(initial_predict)

입력한 값에 대한 H를 리턴해요!
입력한 값에 대한 H를 리턴해요!
[[-6.04032948  0.34726589 -0.9403764  ... 14.83251128 -5.59466042
  -0.97225238]
 [-0.1277486  -0.64623719 17.21786486 ... -6.39279358 -3.36734584
  -7.31494238]
 [-3.8862772  12.1824569  -1.60509282 ... -2.31005266 -2.47345691
  -2.32049342]
 ...
 [-7.53872459 -1.67548626 -3.11262582 ... -2.32601954 -1.1806551
  -1.2335626 ]
 [-3.01522733 -5.56961885 -7.96787756 ... -6.74841453  2.54983649
  -2.02720371]
 [-1.59091121 -4.21687557 -2.31935791 ... -8.22261503 -2.14330608
  -6.05175292]]


In [10]:
### 여기서부터 내가 구현한 부분이라 logic 틀릴 수 있음. ###

# H값에서 예측값 뽑아내기
prediction = np.argmax(initial_predict, axis = 1)

# 라벨값 확인하기
actual = np.argmax(mnist.test.labels, axis = 1)

# 정확도
is_correct = np.equal(prediction, actual)
accuracy = np.mean(is_correct.astype(np.float32))

print(f"정확도는 {accuracy * 100}%입니다.")

정확도는 99.14000034332275%입니다.


# **오늘의 결론**
한 칸에 다 작성하면 아래와 같이 된다

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd

# graph 초기화

tf.reset_default_graph()

# Class 만들기
class CnnModel:
 
    def __init__(self, session, data):
        print("객체 생성") 
        self.sess = session 
        self.mnist = data
        self.build_graph()
 
    # 그래프 그리는 기능
    def build_graph(self):
        print("그래프 그려")

        # placeholder
        self.X = tf.placeholder(shape=[None,784], dtype=tf.float32)
        self.Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
        self.drop_rate = tf.placeholder(dtype=tf.float32)
        
        # convolution layer
        x_img = tf.reshape(self.X, [-1,28,28,1])
        
        L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu)
        L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2], padding="SAME", strides=2)
        L1 = tf.layers.dropout(inputs=L1, rate=self.drop_rate)
        
        L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu)
        L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2], padding="SAME", strides=2)
        L2 = tf.layers.dropout(inputs=L2, rate=self.drop_rate)
        
        L2 = tf.reshape(L2, [-1, 7*7*64])
        
        dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
        dense1 = tf.layers.dropout(inputs=dense1, rate=self.drop_rate)
        
        dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
        dense2 = tf.layers.dropout(inputs=dense2, rate=self.drop_rate)
        
        dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
        dense3 = tf.layers.dropout(inputs=dense3, rate=self.drop_rate)
 
        self.H = tf.layers.dense(inputs=dense3, units=10)

        # cost     
        self.cost = tf.losses.softmax_cross_entropy(self.Y, self.H) 
        
        self.train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)
 
    # 학습시키는 기능
    def train_graph(self):
        self.sess.run(tf.global_variables_initializer())
        print("텐서플로우 그래프 학습")
        num_of_epoch = 30 
        batch_size = 100
        
        for step in range(num_of_epoch):
            num_of_iter = int(self.mnist.train.num_examples / batch_size)
            cost_val = 0
            for i in range(num_of_iter):
                batch_x, batch_y = self.mnist.train.next_batch(batch_size)
                _, cost_val = self.sess.run([self.train, self.cost], feed_dict = {self.X :batch_x,
                                                                                  self.Y : batch_y,
                                                                                  self.drop_rate : 0.4})
            if step % 3 == 0:
                print("cost : {}".format(cost_val))
 
    # H 실행시킨 값 구하기
    def get_Hval(self):
        print("입력한 값에 대한 H를 리턴해요!")
        # 테스트 데이터에 대한 예측값(H)을 구해서 리턴해요!
        return self.sess.run(self.H, feed_dict={self.X : self.mnist.test.images,
                                                self.drop_rate : 0})
# 모델 객체 만들기
mnist = input_data.read_data_sets("/content/drive/My Drive/data/mnist", one_hot = True)
sess = tf.Session()
num_of_model = 10
models = [CnnModel(sess, mnist) for x in range(num_of_model)]

# 모델을 학습해보아요!
for i in range(num_of_model):
    models[i].train_graph()

# test data 넣은 것에 대해 H값을 구해보아요!
initial_predict = np.zeros([mnist.test.num_examples,10]) 

for i in range(num_of_model):
    initial_predict = initial_predict + models[i].get_Hval()
print(initial_predict)

# H값에서 예측값 뽑아내기
prediction = np.argmax(initial_predict, axis = 1)

# 라벨값 확인하기
actual = np.argmax(mnist.test.labels, axis = 1)

# 정확도
is_correct = np.equal(prediction, actual)
accuracy = np.mean(is_correct.astype(np.float32))

print(f"정확도는 {accuracy * 100}%입니다.")

Extracting /content/drive/My Drive/data/mnist/train-images-idx3-ubyte.gz
Extracting /content/drive/My Drive/data/mnist/train-labels-idx1-ubyte.gz
Extracting /content/drive/My Drive/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /content/drive/My Drive/data/mnist/t10k-labels-idx1-ubyte.gz
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
객체 생성
그래프 그려
텐서플로우 그래프 학습
cost : 0.04145151749253273
cost : 0.0312977060675621
cost : 0.011897810734808445
cost : 0.0004356267745606601
cost : 0.004684537183493376
cost : 0.0009076333953998983
cost : 0.0017747249221429229
cost : 0.0011817611521109939
cost : 0.003257737960666418
cost : 0.0006410174537450075
텐서플로우 그래프 학습
cost : 0.08781418949365616
cost : 0.034552156925201416
cost : 0.01623529940843582
cost : 0.04752758890390396
cost : 0.0005183284520171583
cost : 3.9007500163279474e-05
cost : 0.020704658702015877
cost : 5.387692908698227e-06
cost : 0.002130792010575533
cost : 0.0727992802

###### 아래는 코드 짜면서 연습한 부분

In [0]:
# H값에서 예측값 뽑아내기
# print(initial_predict)
# print(initial_predict.shape)
# print(type(initial_predict))
prediction = np.argmax(initial_predict, axis = 1)
print(prediction)

actual = np.argmax(mnist.test.labels, axis = 1)
print(actual)

is_correct = np.equal(prediction, actual)
print(is_correct)
print(is_correct.astype(np.float32))
print(np.mean(is_correct.astype(np.float32)))


tf_prediction = sess.run(tf.argmax(initial_predict, axis = 1))
print(tf_prediction)
tf_actual = sess.run(tf.argmax(mnist.test.labels, axis = 1))
print(tf_actual)
tf_is_correct = sess.run(tf.equal(tf_prediction, tf_actual))
print(tf_is_correct)
tf_accuracy = sess.run(tf.reduce_mean(tf.cast(tf_is_correct, dtype = tf.float32)))
print(tf_accuracy)

[7 2 1 ... 4 5 6]
[7 2 1 ... 4 5 6]
[ True  True  True ...  True  True  True]
[1. 1. 1. ... 1. 1. 1.]
0.9898
[7 2 1 ... 4 5 6]
[7 2 1 ... 4 5 6]
[ True  True  True ...  True  True  True]
0.9898
